In [1]:
pose_vector1 = [240, 30, 40, 532, 194, 329] #(x,y,x,y,...) : model output image keypoints set
pose_vector2 = [250, 40, 50, 542, 204, 339] #(x,y,x,y,...) : 정자세 image keypoints set

### 두 이미지를 고정된 크기로 resize 및 normalization

In [2]:
def img_resize(pose_vector):
    x_max = 0
    y_max = 0
    x_min = 100000
    y_min = 100000
    for i in range(len(pose_vector)):
        if i%2==0: #x 좌표
            if x_max<pose_vector[i]:
                x_max = pose_vector[i]
            if x_min>pose_vector[i]:
                x_min = pose_vector[i]
        else:
            if y_max<pose_vector[i]:
                y_max = pose_vector[i]
            if y_min>pose_vector[i]:
                y_min = pose_vector[i]

    for i in range(len(pose_vector)):
        if i%2==0: #x 좌표
            pose_vector[i] -= x_min
        else:
            pose_vector[i] -= y_min
        print(pose_vector[i])
    
    for i in range(len(pose_vector)):
        if i%2==0: #x 좌표
            if x_max<pose_vector[i]:
                x_max = pose_vector[i]
            if x_min>pose_vector[i]:
                x_min = pose_vector[i]
        else:
            if y_max<pose_vector[i]:
                y_max = pose_vector[i]
            if y_min>pose_vector[i]:
                y_min = pose_vector[i]
                
    img_width = x_max-x_min
    img_height = y_max-y_min
    
    real_width =200
    real_height = 400
    
    wid_portion = real_width/img_width
    hei_portion = real_height/img_height
    
    print("widht:",img_width)
    print("height:",img_height)
    for i in range(len(pose_vector)):
        if i%2 == 0:
            pose_vector[i]=pose_vector[i]*hei_portion
        else:
            pose_vector[i]=pose_vector[i]*wid_portion
    
    return pose_vector

In [3]:
pose_vector1 = img_resize(pose_vector1)
pose_vector2 = img_resize(pose_vector2)

200
0
0
502
154
299
widht: 240
height: 532
200
0
0
502
154
299
widht: 250
height: 542


### 0~1 사이로 정규화(필요에 의해 사용)

In [4]:
def normalization(x):
    min_value = min(x)
    max_value = max(x) 

    return list(map(lambda x: (x-min_value)/(max_value-min_value), x))

In [5]:
nor1 = normalization(pose_vector1)
nor2 = normalization(pose_vector2)

### Similarity 측정

In [6]:
import math
def weightedDistanceMatching(poseVector1, poseVector2, score):
    # test coordinate
    vector1PoseXY = poseVector1
    vector1Confidences = score
    
    # test threshold score sum
    vector1ConfidenceSum = sum(vector1Confidences)
    
    # train coordinate
    vector2PoseXY = poseVector2

    summation1 = 1 / vector1ConfidenceSum
    summation2 = 0
    
    count = 0
    for i in range(len(vector1PoseXY)):
        count += 1
        tempConf = math.floor(i / 2)
        # multiplying each joint's reliability and distance
        tempSum = vector1Confidences[tempConf] * abs(vector1PoseXY[i] - vector2PoseXY[i])
        # Add joint distance by joint count
        summation2 = summation2 + tempSum
    
    # WeightedDistance
    summation = summation1 * summation2
    summation = summation
    
    return summation

score = [1,1,1,1,1,1]
res = weightedDistanceMatching(pose_vector1,pose_vector2,score)

print(res)

5.268466831284852


In [7]:
ans = weightedDistanceMatching(nor1,nor2,score)

### 각도값 측정

In [8]:
import numpy as np
def get_angle(p1 : list, p2 : list ,p3 : list, angle_vec : bool) -> float:
    rad = np.arctan2(p3[1] - p1[1], p3[0] - p1[0]) - np.arctan2(p2[1] - p1[1], p2[0] - p1[0])
    deg = rad * (180 / np.pi)
    if angle_vec:
        deg = 360-abs(deg)
    return abs(deg)


In [9]:
ret1 = get_angle(pose_vector1[:2],pose_vector1[2:4],pose_vector1[4:6],False)
ret2 = get_angle(pose_vector2[:2],pose_vector2[2:4],pose_vector2[4:6],False)

### 위에서 나온 각각의 값을 일정 %로 이용하여 조합한 뒤 총 결과치 측정하면 됨